In [ ]:
using AbstractTrees
using D3Trees
using HTTP
using Sockets
using JSON

# Fix Jupyter issue that prevents logged stack traces to display correctly 
# https://github.com/JuliaLang/IJulia.jl/pull/1045
using Logging
logger = ConsoleLogger(Base.stderr)
Base.CoreLogging.global_logger(logger)

# Infinite tree using the children interface

Available from `include(joinpath(dirname(dirname(pathof(D3Trees))), "test", "binary_abstract_trees.jl"))`

In [ ]:
"""
Binary tree node. Must have field id.
"""
abstract type BTNode end

bt_depth(id::Int) = floor(UInt, log2(id))
bt_children_ids(id::Int) = [2*id, 2*id+1]
bt_depth(n::BTNode) = bt_depth(n.id)
Base.show(io::IO, n::BTNode) = print(io, "$(n.id) (d=$(bt_depth(n))) -> $(getfield.(AbstractTrees.children(n), :id))")
D3Trees.style(BTNode) = "fill:yellow"

"""
    LimitedDepthTree(id, max_leaf_depth)

Create binary tree rooted at index with leave depth specified by the `max_leaf_depth` parameter.
The AbstractTrees.children method does not expand the tree beyond the `max_leaf_depth`.
Maximum `max_leaf_depth` is typemax(Int)
"""
struct LimitedDepthTree <: BTNode
    id::Int
    max_leaf_depth::Int

    function LimitedDepthTree(id, leaf_depth)
        @assert id>0 "All notes must have id > 0, root has 1."
        @assert leaf_depth>=0
        new(id, leaf_depth)
    end
end

LimitedDepthTree(;root_id=1, max_leaf_depth=typemax(Int)) =  LimitedDepthTree(root_id, max_leaf_depth)
expand(n::LimitedDepthTree) = bt_depth(n) < n.max_leaf_depth


# The required interface method
AbstractTrees.children(n::LimitedDepthTree) = expand(n) ? LimitedDepthTree.(bt_children_ids(n.id), n.max_leaf_depth) : LimitedDepthTree[]

# Lazy loading infinite tree
 - F12 to see the javascript logging
 - If it does not appear on its own, run any cell below to view the Julia log
 - If developing remotely, you might have to tunnel the tree server port, e.g. with `ssh -A -L 16370:localhost:16370 yourname@yourserver.eu`

In [ ]:
# Set logging level to debug to see the server communication
# debuglogger = ConsoleLogger(stderr, Logging.Debug)
# Base.CoreLogging.global_logger(debuglogger)

In [ ]:
# Tree with very large depth
ldroot = LimitedDepthTree()

# lazy_expand_after_depth controls the initial generation of the tree, 
# through the `children` AbstractTrees.jl interface
t = D3Tree(ldroot, lazy_expand_after_depth=0, init_expand=1, lazy_subtree_depth=1)

In [ ]:
# run another tree of the same server
Base.show(io::IO, n::BTNode) = print(io, "") # this also affects new nodes of previous tree
D3Tree(ldroot, lazy_expand_after_depth=0, init_expand=1, lazy_subtree_depth=1)

In [ ]:
# run another tree on new server
D3Tree(ldroot, lazy_expand_after_depth=0, init_expand=1, lazy_subtree_depth=1, port=4564)

In [ ]:
# Show existing servers
D3Trees.SERVERS

In [ ]:
# Show tree data on default server
# D3Trees.SERVERS[D3Trees.DEFAULT_PORT].tree_data

# Reset default server
# D3Trees.reset_server(D3Trees.DEFAULT_PORT)

# The data is removed
# D3Trees.SERVERS[D3Trees.DEFAULT_PORT].tree_data

# Reset all servesrs
# D3Trees.reset_server!()

# Shutdown all servers
D3Trees.shutdown_server!()